#Import librerie

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2


#Upload del modello .h5

In [ ]:
from google.colab import files
uploaded = files.upload()   # seleziona modello_pneumonia.h5
model = tf.keras.models.load_model("modello_pneumonia.h5")


#Upload immagine radiografica

In [ ]:
# Carica l'immagine radiografica caricata dal PC
img_path = "stock-photo-x-ray-photo-image-of-chest-area-2569557853.jpg"
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0


#Predizione

In [ ]:
preds = model.predict(img_array)
prob = preds[0][0]
diagnosi = "Pneumonia" if prob > 0.5 else "Normal"
print(f"Classe predetta: {diagnosi} (probabilità: {prob:.2f})")


In [ ]:
# Grad-CAM
model.summary()  # controlla il nome dell'ultimo Conv2D (es. conv2d_2)
grad_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=[model.get_layer("conv2d_2").output, model.output]
)

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(img_array)
    loss = predictions[:, np.argmax(predictions[0])]

grads = tape.gradient(loss, conv_outputs)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

conv_outputs = conv_outputs[0]
heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
heatmap = tf.squeeze(heatmap)
heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

# Sovrapponi la heatmap all'immagine originale
img_cv = cv2.imread(img_path)
heatmap = cv2.resize(heatmap.numpy(), (img_cv.shape[1], img_cv.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img_cv

plt.imshow(cv2.cvtColor(superimposed_img.astype("uint8"), cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title(f"Diagnosi AI: {diagnosi} (probabilità: {prob:.2f})")
plt.show()

# Interpretazione per radiologo
print("\n👨‍⚕️ Spiegazione tecnica per radiologo:")
print(f"Il modello ha classificato l'immagine come '{diagnosi}' con probabilità {prob:.2f}.")
print("La heatmap evidenzia le aree polmonari che hanno maggiormente influenzato la decisione.")
print("Il radiologo può verificare se l'AI si è concentrata su zone patologiche (consolidamenti, opacità) o su artefatti.")

# Interpretazione semplificata per paziente
print("\n🧑‍⚕️ Spiegazione semplificata per paziente:")
if diagnosi == "Pneumonia":
    print("L'analisi dell'AI suggerisce la presenza di polmonite.")
    print("La mappa colorata mostra le zone dei polmoni che hanno portato a questa conclusione.")
else:
    print("L'analisi dell'AI non ha rilevato segni di polmonite.")
    print("La mappa colorata mostra le zone dei polmoni che sono state analizzate.")